## DM 2023 tutorial: scale and pitch

Shuxin Meng

Part of the content is taken from previous years' tutorials by Gabriele Cecchetti, Johannes Hentschel, Christoph Finkensiep, and Daniel Harasim

In [ ]:
import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from fractions import Fraction
from collections import defaultdict, Counter
from iteration_utilities import deepflatten #flatten nested lists

from music21 import midi, note, interval, pitch, serial, stream, instrument, meter, key, converter, scale, graph
import itertools
import random
from ast import literal_eval

# import cufflinks as cf
# cf.go_offline()
# cf.set_config_file(theme='white')
# pd.options.plotting.backend = "plotly"

# import nltk #for NLP
# import string
# import mchmm #for Markov models

# Helpers

In [ ]:
def play(stream):
    """Shortcut to play a stream"""
    midi.realtime.StreamPlayer(stream).play()
    
def get_stream(pitches, **kwargs):
    """Turns a collection of pitches into a Stream."""
    st = stream.Stream(**kwargs)
    if isinstance(pitches[0], pitch.Pitch):
        for p in pitches:
            st.append(note.Note(p))
    else:
        for p in pitches:
            st.append(p)
    return st

def get_measure(pitches, dur=1, **kwargs):
    """Turns a collection of pitches into a stream.Measure"""
    st = stream.Measure(**kwargs)
    if isinstance(pitches[0], pitch.Pitch):
        for p in pitches:
            st.append(note.Note(p, quarterLength=dur))
    else:
        for p in pitches:
            st.append(p)
    return st

def get_notes(pitches, **kwargs):
    """Convert a collection of pitches into a list of Notes"""
    return [note.Note(p, **kwargs) for p in pitches]


In many musical cultures, the organisation of pitch is based on the selection of an alphabet of pitch material. In most of the Western classical tradition, the largest alphabet is formed by the *chromatic* collection, comprising a subdivision of the octave in 12 chromatic steps, the *semitones*. 

In [ ]:
pitch.Pitch(84).name

In [ ]:
CHROMATIC = range(60, 60+13)
m = stream.Measure()
m.append([note.Note(pitch = x) for x in CHROMATIC])
m.show()

Intervals among pitches are computed by counting the number of 'steps' that separate them.

# Pitch class and Pitch-class sets

Starting from the chromatic collection, other pitch collections can be formed as subsets of the chromatic one. Note that octave equivalence induces a partition of the chromatic collection (which, in principle, extends indefinitely to the right and to the left), into 12 distinct pitch-classes. E.g.:

In [ ]:
note.Note("F8").pitch.pitchClass == note.Note("F4").pitch.pitchClass

The interval between two pitch classes is not well-defined: for example, if we consider pitch-classes C and G#, we can have the G# lying 8 semitones above the C...

In [ ]:
C = note.Note('C4')
highGis = note.Note('G#4')
ChighGis = interval.Interval(noteStart = C, noteEnd = highGis)
ChighGis.semitones

or the one 4 semitones below the C:

In [ ]:
lowGis = note.Note('G#3')
lowGisC = interval.Interval(noteStart = lowGis, noteEnd = C)
lowGisC.semitones

We define instead the interval class between two pitch classes as the smallest interval between representatives of the two classes:

In [ ]:
n1 = note.Note('C')
n2 = note.Note('G#')
n1n2 = interval.Interval(noteStart = n1, noteEnd = n2)
n1n2.intervalClass

Pitch-class sets are unordered collections of pitch classes. Each pitch class in a pitch-class set may be identified by a number from 0 to 11, 0 corresponding to the pitch class of a C. 

In [ ]:
pitchClassSet = {0, 3, 7}

def notes_from_pitchClassSet(pitchClassSet, transpose = 0):
    """Returns list of notes based on a pitchClassSet and a transposition (default, start on C)"""
    return [note.Note(pitch = pitchClass+transpose) for pitchClass in pitchClassSet]

pitchClassSetNotes = notes_from_pitchClassSet(pitchClassSet)
pitchClassSetNotes

In order to characterize a pitch class set, we can look at the intervals formed by the members of the set. In particular, we consider all possible interval classes between any two members of the set. The interval vector of a pitch class is a 6-vector whose i-th component reflects how many interval classes of size i are to be found among all the possible pairings of elements of the set.

In [ ]:
def intervalVector(notes):
    """Compute the interval vector of a list of notes"""
    vector = {i+1:0 for i in range(6)}
    pairs = itertools.combinations(notes, 2)
    for pair in pairs:
        intClass = interval.Interval(pair[0], pair[1]).intervalClass
        vector[intClass] += 1
    return vector

intervalVector(pitchClassSetNotes)


Interval vectors capture an important aspect of the compositional possibilities that arise when adopting a certain pitch-class set as the pitch alphabet for a piece (or a portion thereof): they tell us what are the possible intervals that can be formed if those pitches are played one after the other (melodically) or simultaneously (harmonically).

Algebraic properties of pitch-class sets, such as their invariances under transformations, are a fertile terrain for creative and algorithmic exploration. If you are interested, you can read more on musical set theory, e.g. starting from this simple introduction https://musictheory.pugetsound.edu/mt21c/SetTheory.html.


In [ ]:
def select_from_chromatic(intervals, root = 'C'):
    """Selects pitches from a chromatic collection based on a set of intervals"""
    return [note.Note(pitch = pitch.Pitch(root).midi + x) for x in [0]+[sum(intervals[:i+1]) for i in range(len(intervals))]]

# Scales and modes

A very commong ordered subcollection of the chromatic collection in Western classical music is the major scale, that we can identify by indicating the intervals in semitones between consecutive tones:

In [ ]:
MAJOR = [2, 2, 1, 2, 2, 2, 1]
m = stream.Measure()
m.append(select_from_chromatic(MAJOR))
play(m)
m.show()

Get the interval name from distance in semitones

In [ ]:
MAJOR_INTERVAL = [interval.Interval(i).name for i in MAJOR]
MAJOR_INTERVAL

Note that the intervals in a major scale are not all the same: between the third and the fourth note, as well as between the seventh and the eigth (which is equivalent to the first under octave equivalence) there is an interval of just 1 semitone. 

We can obtain other *modes* that are based on the same selection of itnervals by rotating the list of intervals, and changing the root.

In [ ]:
def rotate_list(l, n):
    """rotate list l by n steps"""
    return l[n:] + l[:n]

def select_mode(scale, mode, root):
    """select a mode based on a root"""
    return select_from_chromatic(rotate_list(scale, mode), root)


## Exercise:

How to get the D major sequence?

In [ ]:
select_mode(MAJOR, 0, "D")

For example, here is the so-called "phrygian" mode, which shares all pitches with the major scale, but with a different ordering of intervals (starting from root E):

In [ ]:
phrygian = select_mode(MAJOR, 2, 'E')
m = stream.Measure()
m.append(phrygian)
play(m)
m.show()

## Exercise: 

How to get back the interval structure of phrygian by only looking at the phrygian

The expected output should be: [1, 2, 2, 2, 3, 2, 2]

In [ ]:
[(pitch.Pitch(t.name).midi - pitch.Pitch(s.name).midi) % 6 for s, t in zip(phrygian, phrygian[1:])]

Let's look at the pentatonic scales

In [ ]:
PENTATONIC = [2, 2, 3, 2, 3]

In [ ]:
m = stream.Measure()
m.append(select_from_chromatic(PENTATONIC))
play(m)
m.show()

## Exercise: 

There are five versions of this pentatonic: C D E G A, D E G A C, E G A C D, G A C D E, A C D E G

How to get the five modes in a pentatonic starting from C? 


In [ ]:
[select_mode(PENTATONIC, i, pitch.Pitch(60).name) for i in range(len(PENTATONIC) - 1)]

How many pentatonic in a chromatic set of 12 pitches?

In [ ]:
pentatonic_all = []
for i in CHROMATIC:
    pentatonic_all.append([select_mode(PENTATONIC, j, i) for j in range(len(PENTATONIC))])
len(list(deepflatten(pentatonic_all, depth = 1)))

In [ ]:
A_PENTATONIC = pentatonic_all[0][4]

# Melody generation
## Random

In [ ]:
get_measure(A_PENTATONIC).show()

In [ ]:
sc = scale.ConcreteScale(pitches = A_PENTATONIC)

In [ ]:
pitch_collection = sc.getPitches('C4', 'C5')
print(f"pitch_collection contains {len(pitch_collection)} pitches.")

## Exercise:
Can you generate a random melody with 20 notes?

In [ ]:
random_melody = random.choices(pitch_collection, k = 20)
get_measure(random_melody).show()

## Exercise:
Can you add duration for each note with either quarter or eighth length?

In [ ]:
notes = [note.Note(pitch=p, quarterLength = random.choice([0.5, 1])) for p in random_melody]
melody = get_stream(notes)
melody.show()
play(melody)

# Examine the dataset

In [ ]:
data_essen = pd.read_csv('Essen_folksong_collection.csv', index_col=0, sep='\t')
data_essen["DGIs"] = data_essen["DGIs"].apply(literal_eval)
data_essen["spelled_pitches"] = data_essen["spelled_pitches"].apply(literal_eval)

data_essen["region"].unique()

In [ ]:
essen_china = data_essen[(data_essen["region"] == 'han') & (data_essen["key"].notnull())]
essen_china.head()

In [ ]:
sequences_essen_key = essen_china["key"].to_list()
len(sequences_essen_key)

In [ ]:
sequences_essen_root = [i.split()[0]  for i in sequences_essen_key ]
len(sequences_essen_root)

In [ ]:
sequences_essen_mode = [i.split()[1]  for i in sequences_essen_key ]
len(sequences_essen_mode)

In [ ]:
# sequences_essen_int = [
#     [ interval.strip( '\"[\',\]' ) for interval in row.DGIs.split() ] 
#     for (i, row) in essen_china.iterrows()
# ]

# len(sequences_essen_int)

In [ ]:
# split and strip pitches
sequences_essen_pitch = []
for (i, row) in essen_china.iterrows():
    temp = []
    for p in row.spelled_pitches:
#         temp.append(pitch.Pitch(p).name) # if only consider pitch class
        temp.append(p) # with octave
    
    sequences_essen_pitch.append(temp)

len(sequences_essen_pitch)

In [ ]:
sequences_essen_pitch_transposed = []
for p in range(len(sequences_essen_pitch)):
    temp = []
    for n in sequences_essen_pitch[p]:
        temp.append(pitch.Pitch(n).transpose(interval.Interval(pitch.Pitch(sequences_essen_root[p]), pitch.Pitch("C"))))
    
    sequences_essen_pitch_transposed.append(temp)
len(sequences_essen_pitch_transposed)

In [ ]:
pd.Series(deepflatten(sequences_essen_pitch, depth = 1)).apply(lambda x : pitch.Pitch(x).pitchClass).value_counts(normalize=True).sort_index().plot(kind='bar', color="orange")

In [ ]:
pd.Series(deepflatten(sequences_essen_pitch_transposed, depth = 1)).apply(lambda x : pitch.Pitch(x).pitchClass).value_counts(normalize=True).sort_index().plot(kind='bar', color="orange")

## 1. Computing bigrams

In [ ]:
def bigrams_seq(seq):
    return list(zip(seq[:-1], seq[1:]))

def bigrams_corpus(seqs):
    return [bg for seq in seqs for bg in bigrams_seq(seq)]

bigrams = bigrams_corpus(sequences_essen_pitch)
bg_counts = Counter(bigrams)
bg_counts.most_common(20)

## 2. Draw the bigrams

In [ ]:
x1s    = [ x1  for ((x1, x2), count) in bg_counts.items() ]
x2s    = [ x2  for ((x1, x2), count) in bg_counts.items() ]
counts = [ count for ((x1, x2), count) in bg_counts.items() ]

# counts
# bg_counts

In [ ]:
df = pd.DataFrame({"x1": [x for x in x1s], "x2": [i for i in x2s], "counts": counts})
df.head()

df_wide = df.pivot_table(index=["x1"], columns="x2", values="counts").fillna(0)
df_wide.head()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(data=df_wide, cmap="coolwarm")

In [ ]:
# Compute transition table

transitions = dict()
for ((x1, x2), count) in bg_counts.items():
    if not (x1 in transitions):
        transitions[x1] = dict()
    transitions[x1][x2] = count

In [ ]:
def normalize_dict(dictionary):
    S = sum(dictionary.values())
    return dict([(k, v/S) for (k, v) in dictionary.items()])

In [ ]:
transitions_norm = dict([(x1, normalize_dict(counts)) for (x1, counts) in transitions.items()])

In [ ]:
# transitions_norm

In [ ]:
def next_note(trans, note):
    dist = trans[note]
    items = dist.items()
    candidates = [note for (note, prob) in items]
    probs = [prob for (note, prob) in items]
    return np.random.choice(candidates, p = probs)

In [ ]:
def sample_piece(trans, length):
    fist_note = np.random.choice(list(trans.keys()))
    notes = [fist_note]
    for i in range(1, length):
        note = next_note(trans, fist_note)
        notes.append(note)
    return notes

In [ ]:
notes = [note.Note(pitch = p, quarterLength = 0.5) for p in sample_piece(transitions_norm, 20)]


In [ ]:
random_china = get_measure(notes)
random_china.show()
play(random_china)